<a href="https://colab.research.google.com/github/emmab-collab/cerebral-aneurysm-detection-3d/blob/main/notebooks/00_fonctions_utiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from scipy.ndimage import rotate, shift,zoom , map_coordinates, gaussian_filter
import random
import os
import glob
import ast

import matplotlib.pyplot as plt
import pydicom

from tqdm import tqdm

# **1. Accès à des infos**

## **1. Instance Number du centre**

In [ ]:
def get_instance_number(patient_path):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]
    exemple_path = os.path.join(series_path,
                                  f"{numero_patient}/{numero_coupe}.dcm")
    #print('Récupération des coordonnées du voxel...')
    ds = pydicom.dcmread(exemple_path)

    InstanceNumber=ds.InstanceNumber

    return int(InstanceNumber)

## **2. Coordonnée z du centre**

In [ ]:
def coordonnee_z(patient_path,InstanceNumber=163):

    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]
    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))

    # Trouver l’indice z dans le volume
    z_index = [i for i, s in enumerate(slices) if int(s.InstanceNumber) == InstanceNumber][0]

    return z_index

## **3. Coordonnées du centre**

In [ ]:
def get_center(series_path,patient_path,df_loc):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]

    InstanceNumber=get_instance_number(patient_path)
    z=coordonnee_z(patient_path,InstanceNumber)

    coord_str = df_loc[df_loc['SOPInstanceUID'] == numero_coupe]['coordinates'].iloc[0]
    coord_dict = ast.literal_eval(coord_str)
    x = coord_dict['y']
    y = coord_dict['x'] #fait exprès

    center=np.array([x,y,z])
    return center

## **4. Pixel Spacing et Slice Thickness**

In [ ]:
def get_pixelspacing(path):
    """
    Récupère l'espacement voxel (mm) depuis un fichier DICOM.
    Retourne (row_spacing, col_spacing, slice_thickness)
    """
    dcm = pydicom.dcmread(path)

    # Espacement dans le plan (mm/pixel)
    if "PixelSpacing" in dcm:
        row_spacing, col_spacing = [float(x) for x in dcm.PixelSpacing]
    else:
        row_spacing, col_spacing = None, None

    # Épaisseur de coupe (mm)
    slice_thickness = float(getattr(dcm, "SliceThickness", 1.0))

    return row_spacing, col_spacing, slice_thickness

## **5. Patient_ID**

In [ ]:
def get_patient_ID(patient_path):
    return str(os.path.basename(patient_path))

## **6. Position de l'anévrisme : np.array (1,13)**

In [ ]:
def get_position(df_train,patient_path):
    positions=['Left Infraclinoid Internal Carotid Artery',
       'Right Infraclinoid Internal Carotid Artery',
       'Left Supraclinoid Internal Carotid Artery',
       'Right Supraclinoid Internal Carotid Artery',
       'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery',
       'Anterior Communicating Artery', 'Left Anterior Cerebral Artery',
       'Right Anterior Cerebral Artery', 'Left Posterior Communicating Artery',
       'Right Posterior Communicating Artery', 'Basilar Tip',
       'Other Posterior Circulation']
    row=df_train[df_train["SeriesInstanceUID"] == get_patient_ID(patient_path)][positions].values.flatten()
    return row

# **2. Visualisation d'image**

#définir notre dataframe
df_loc=pd.read_csv("/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv")
#lien de la série
series_path='/kaggle/input/rsna-intracranial-aneurysm-detection/series'
#lien du patient
patient_path=os.path.join(series_path,
                                  f"{df_loc['SeriesInstanceUID'][35]}")
#lien d'une coupe
exemple_path = os.path.join(series_path,
                                  f"{df_loc['SeriesInstanceUID'][35]}/{df_loc['SOPInstanceUID'][35]}.dcm")
ds = pydicom.dcmread(exemple_path)

# accéder aux pixels
image = ds.pixel_array
image.shape

In [ ]:
def show_middle_slices(volume):
    # volume shape : (X, Y, Z)

    mid_x = volume.shape[0] // 2
    mid_y = volume.shape[1] // 2

    mid_z = volume.shape[2] // 2


    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    # Coupe axiale (XY plane à profondeur z)
    axes[0].imshow(volume[:, :, mid_z].T, cmap='gray')  # transpose pour que X horizontal, Y vertical
    axes[0].set_title(f'Axiale (z={mid_z})')
    axes[0].axis('on')

    # Coupe coronale (XZ plane à coordonnée y)
    axes[1].imshow(volume[:, mid_y, :].T, cmap='gray')  # transpose pour X horizontal, Z vertical
    axes[1].set_title(f'Coronale (y={mid_y})')
    axes[1].axis('on')

    # Coupe sagittale (YZ plane à coordonnée x)
    axes[2].imshow(volume[mid_x, :, :].T, cmap='gray')  # transpose pour Y horizontal, Z vertical
    axes[2].set_title(f'Sagittale (x={mid_x})')
    axes[2].axis('on')

    plt.tight_layout()
    plt.show()

In [ ]:
def show_slice_with_point(volume, coord, plane="axial"):
    x, y, z = coord.astype(int)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    if plane == "axial":       # plan XY à profondeur z
        img = volume[:, :, z].T
        ax[0].imshow(img, cmap="gray")
        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(x, y, c="r", s=40, marker="x")
        title = f"Axial z={z}"

    elif plane == "sagittal":  # plan YZ à abscisse x
        img = volume[x, :, :].T
        ax[0].imshow(img, cmap="gray")
        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(y, z, c="r", s=40, marker="x")

        title = f"Sagittal x={x}"

    elif plane == "coronal":   # plan XZ à ordonnée y
        img = volume[:, y, :].T
        ax[0].imshow(img, cmap="gray")
        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(x, z, c="r", s=40, marker="x")
        title = f"Coronal y={y}"

    else:
        raise ValueError("plane doit être 'axial', 'sagittal' ou 'coronal'")

    for a in ax:
        a.set_title(title)
        a.axis("on")
    plt.tight_layout()
    plt.show()

# **3. Manipulation de dataset**

In [ ]:
def ajouter_Modality(df_main,df_info):
    df_merged = df_main.merge(
        df_info[['SeriesInstanceUID', 'Modality']],
        on='SeriesInstanceUID',
        how='left'  # conserve toutes les lignes de df_main
    )
    return df_merged

# **4. Preprocessing**

## **1. Chaine de preprocessing d'images**

In [ ]:
def dicom_to_numpy(patient_path):

    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]

    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))


    # On empile les pixel_array en un volume 3D NumPy (X,Y,Z)
    target_shape = slices[0].pixel_array.shape

    slices = [s for s in slices if s.pixel_array.shape == target_shape]
    volume = np.stack([s.pixel_array for s in slices], axis=-1)

    # Récupération du spacing réel
    pixel_spacing = slices[0].PixelSpacing
    dx, dy = pixel_spacing
    dz = getattr(slices[0], 'SliceThickness', 1.0)  # fallback si manquant

    return volume, (dx,dy,dz)

def resample(volume, spacing, target_spacing=(0.4, 0.4, 0.4)):
    zoom_factors = [s / t for s, t in zip(spacing, target_spacing)]
    new_volume = zoom(volume, zoom_factors, order=1)
    return new_volume

def crop(volume):

    # On crée un masque des voxels non nuls
    mask = volume > (volume.max() * 0.1)
    if not mask.any():
        return volume  # rien à couper

    # On récupère les indices min/max pour chaque dimension
    x_min, x_max = mask.any(axis=(1,2)).nonzero()[0][[0, -1]] #axe_x
    y_min, y_max = mask.any(axis=(0,2)).nonzero()[0][[0, -1]] #axe_y
    z_min, z_max = mask.any(axis=(0,1)).nonzero()[0][[0, -1]] #axe_z

    # Crop
    cropped = volume[x_min:x_max+1, y_min:y_max+1, z_min:z_max+1]
    return cropped, (x_min,y_min,z_min)

def normalization(volume):

    v_min, v_max = volume.min(), volume.max()
    if v_max > v_min:  # éviter la division par zéro
        volume = (volume - v_min) / (v_max - v_min)
    else:
        volume = np.zeros_like(volume)
    return volume

## **2. Resample des coordonnées du centre**

In [ ]:
def resample_coordonnees(spacing, coords, target_spacing=0.4):
    x,y,z=coords
    if isinstance(target_spacing, (int, float)):
        target_spacing = (target_spacing, target_spacing, target_spacing)

    # coord physiques
    coords_mm = np.array([x*spacing[0], y*spacing[1], z*spacing[2]])
    # nouveaux indices
    new_voxel = coords_mm / np.array(target_spacing)
    return new_voxel

## **3. Chaine de preprocessing du volume et des coordonnées**

In [ ]:
def preprocessing_volume_and_coords(series_path,patient_path,df_loc):

    coords=get_center(series_path,patient_path,df_loc)
    volume,spacing = dicom_to_numpy(patient_path)

    resample_volume = resample(volume, spacing,target_spacing=(0.4, 0.4, 0.4))
    resample_coords=resample_coordonnees(spacing, coords,target_spacing=0.4)

    crop_volume, crop_indices = crop(resample_volume)
    crop_coords=resample_coords - crop_indices

    norm_volume = normalization(crop_volume)

    return norm_volume,crop_coords

In [ ]:
## Pour la fonction d'inférence
def preprocessing_volume(patient_path):

    volume,spacing = dicom_to_numpy(patient_path)
    resample_volume = resample(volume, spacing,target_spacing=(0.4, 0.4, 0.4))
    crop_volume, crop_indices = crop(resample_volume)
    new_volume = normalization(crop_volume)

    return new_volume

# **5. Data augmentation**

## **1. Random deformation d'un volume**

In [ ]:
def random_deformation(volume, grid_size=3, max_displacement=3):

    shape = volume.shape
    assert len(shape) == 3, "Le volume doit être 3D"


    # Coordonnées originales du volume
    dz, dy, dx = shape
    z, y, x = np.meshgrid(
        np.linspace(0, dz-1, shape[0]),
        np.linspace(0, dy-1, shape[1]),
        np.linspace(0, dx-1, shape[2]),
        indexing="ij"
    )

    # Génère une grille de points de contrôle (par ex. 3x3x3)
    grid_z = np.linspace(0, dz-1, grid_size)
    grid_y = np.linspace(0, dy-1, grid_size)
    grid_x = np.linspace(0, dx-1, grid_size)
    control_points = np.meshgrid(grid_z, grid_y, grid_x, indexing="ij")

    # Crée un champ de déplacements aléatoires
    # (même taille que la grille de contrôle)
    displacement = [
        np.random.uniform(-max_displacement, max_displacement, size=(grid_size,grid_size,grid_size))
        for _ in range(3)
    ]

    # Interpolation spline cubique du champ de déplacement

    from scipy.interpolate import RegularGridInterpolator
    disp_interp = [
        RegularGridInterpolator((grid_z, grid_y, grid_x), d, bounds_error=False, fill_value=0)
        for d in displacement
    ]

    # Calcule le champ de coordonnées déformées
    coords = np.array([z, y, x])
    dz_new = disp_interp[0](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)
    dy_new = disp_interp[1](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)
    dx_new = disp_interp[2](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)

    z_new = z + dz_new
    y_new = y + dy_new
    x_new = x + dx_new

    # Applique la déformation au volume
    deformed = map_coordinates(volume, [z_new, y_new, x_new], order=3, mode='reflect')

    return deformed

## **2. Data augmentation d'un volume**

In [ ]:
def data_augmentation(volume,n_aug):

    augmented_volumes = []
    for i in range(n_aug):
        vol_def = random_deformation(volume)
        augmented_volumes.append(vol_def)

    augmented_volumes = np.stack(augmented_volumes)  # shape = (12, 48, 48, 48)
    return augmented_volumes

## **3. Création d'un dataset augmenté**

In [ ]:
def dataset_augmented(liste_volumes,n_aneurysm):
    liste_cubes = []

    for i in tqdm(range(len(liste_volumes))):
        volume = liste_volumes[i]  # shape (48,48,48)

        # data_augmentation doit renvoyer (n, 48, 48, 48)
        aug = data_augmentation(volume,n_aneurysm)

        liste_cubes.append(aug)

    # Concaténer tous les résultats en un seul tableau
    liste_cubes = np.concatenate(liste_cubes, axis=0)  # shape (N*n, 48, 48, 48)
    return liste_cubes

# **6. Découpe de cubes**

## **Sur un df positif**

## **1. Découpe de N cubes comportant l'anévrisme**

In [ ]:
def extract_positives_cubes(volume, aneurysm_coords, N=10, cube_size=48, max_shift=7):

    cubes = []
    x_c, y_c, z_c = aneurysm_coords
    x_c, y_c, z_c = int(x_c), int(y_c), int(z_c)

    # Dimensions du volume
    D, H, W = volume.shape

    for _ in range(N):
        # Déplacement aléatoire autour de l'anévrisme
        dx = np.random.randint(-max_shift, max_shift+1)
        dy = np.random.randint(-max_shift, max_shift+1)
        dz = np.random.randint(-max_shift, max_shift+1)

        # Centre du cube
        cx, cy, cz = x_c + dx, y_c + dy, z_c + dz


        # Limiter les bornes pour rester dans le volume
        x_start = max(0, cx - cube_size//2)
        y_start = max(0, cy - cube_size//2)
        z_start = max(0, cz - cube_size//2)

        x_end = min(D, x_start + cube_size)
        y_end = min(H, y_start + cube_size)
        z_end = min(W, z_start + cube_size)

        # Ajuster le start si on est proche de la limite
        x_start = x_end - cube_size
        y_start = y_end - cube_size
        z_start = z_end - cube_size

        # Extraire le cube
        cube = volume[x_start:x_end, y_start:y_end, z_start:z_end]
        cubes.append(cube)

    return np.array(cubes)  # shape : (N, cube_size, cube_size, cube_size)


## **2. Découpe de N cubes négatifs probablement proches de l'anévrisme**

In [ ]:
import numpy as np
def extract_negative_cubes(
    volume: np.ndarray,
    aneurysm_coords: np.ndarray,
    N: int = 20,
    cube_size: int = 48,
    min_distance: int = 75,
    exp_scale: float = 5.0,
    max_trials: int = 10_000):

    D, H, W = volume.shape
    half = cube_size // 2
    x_c, y_c, z_c = map(float, aneurysm_coords)

    cubes = []
    centers = []
    trials = 0

    while len(cubes) < N and trials < max_trials:
        trials += 1


        # Tirer une distance radiale : min_distance + Exp(scale)
        r = min_distance + np.random.exponential(exp_scale)

        # Tirer direction aléatoire sur la sphère
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        dx = r * np.sin(theta) * np.cos(phi)
        dy = r * np.sin(theta) * np.sin(phi)
        dz = r * np.cos(theta)

        cx = int(round(x_c + dx))
        cy = int(round(y_c + dy))
        cz = int(round(z_c + dz))

        # Vérifier que le cube tient dans le volume
        if (half <= cx < D - half and
            half <= cy < H - half and
            half <= cz < W - half):


            cube = volume[
                cx - half : cx + half,
                cy - half : cy + half,
                cz - half : cz + half
            ]

            cubes.append(cube)
            centers.append((cx, cy, cz))

    if len(cubes) < N:
        raise RuntimeError(
            f"Seulement {len(cubes)} cubes trouvés après {trials} essais."
        )

    return np.stack(cubes), centers

## **3. Création d'un dictionnaire patient avec cubes positifs et négatifs**


``cubes (50, 48, 48, 48)``
``patient_ID (50, 1)``
``labels (50, 1)``
``positions (50, 13)``


In [ ]:
def dictionnaire_patient(series_path,patient_path,df,df_train):
    data={}

    volume,aneurysm_coords=preprocessing_volume_and_coords(series_path,patient_path,df)

    positive_cubes=extract_positives_cubes(volume, aneurysm_coords, N=25, cube_size=48, max_shift=20)
    negative_cubes= extract_negative_cubes(volume,aneurysm_coords,N=25,cube_size=48,min_distance=75,exp_scale=5.0)[0]


    position_single = get_position(df_train,patient_path).reshape(1,13)

    cubes = np.concatenate([positive_cubes,negative_cubes],axis=0)
    patient_ID = np.array([get_patient_ID(patient_path)] * (positive_cubes.shape[0] + negative_cubes.shape[0])).reshape(-1, 1)
    labels = np.concatenate([np.ones((positive_cubes.shape[0], 1), dtype=int), np.zeros((negative_cubes.shape[0], 1), dtype=int)])
    positions = np.concatenate([
        np.repeat(position_single,positive_cubes.shape[0] , axis=0),
        np.zeros((negative_cubes.shape[0], 13))
    ],axis=0)

    # Créer le dictionnaire par patient
    data_patient = {
        "cubes": cubes,
        "patient_ID": patient_ID,
        "labels": labels,
        "positions": positions
    }
    return data_patient

## **4. Création d'un dictionnaire avec tous les patients**

In [ ]:
def build_dataset_dict(series_path, df, df_train):

    data = {}

    for i in tqdm(range(len(df))):
        series_uid = df.iloc[i]['SeriesInstanceUID']
        patient_path = os.path.join(series_path, series_uid)

        try:
            data[f'patient_{i}'] = dictionnaire_patient(series_path, patient_path, df, df_train)
        except Exception as e:
            print(f"Erreur pour {series_uid}: {e}")
            continue  # passer au patient suivant

    return data

## **Sur un df négatif**

## **1. Découpe de cubes non recouvrants**

In [ ]:
def get_non_overlapping_cubes(volume, cube_size=48):

    z_dim, y_dim, x_dim = volume.shape
    cubes = []

    # parcours par pas de cube_size
    for zi in range(0, z_dim - cube_size + 1, cube_size):
        for yi in range(0, y_dim - cube_size + 1, cube_size):
            for xi in range(0, x_dim - cube_size + 1, cube_size):
                cube = volume[zi:zi+cube_size, yi:yi+cube_size, xi:xi+cube_size]
                cubes.append(cube)

    return cubes

# **7. Prédiction**

In [ ]:
def predict_cube(cube, model, threshold=0.5, device=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()

    # convertir en tensor et ajouter channel et batch
    cube_tensor = torch.tensor(cube, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(cube_tensor)       # shape (1,)
        prob = torch.sigmoid(logits).item()

    label = 1 if prob > threshold else 0

    return prob, label

# **8. Détection des erreurs**

## **Sur un df positif**

In [ ]:
def get_erreurs(df):
    faux_negatif=[]
    probas_faux_negatif=[]
    faux_positif=[]
    probas_faux_positif=[]

    for i in tqdm(range(len(df))):
        patient_path=os.path.join(series_path,
                                      f"{df['SeriesInstanceUID'][i]}")
        coords = get_center(patient_path)
        volume, center = preprocessing_volume_and_coords(patient_path,coords)
        cubes=crop_non_overlapping(volume, center, size=48, n_close=25, n_random=25)

        if not cubes:

            continue

        #cubes["aneurysm"]:


        prob, label=predict_cube(cubes["aneurysm"], model, threshold=0.5, device=None)
        if label==0 :
            faux_negatif.append(cubes["aneurysm"])
            probas_faux_negatif.append(prob)

        for cube in cubes["closest"]:
            prob, label=predict_cube(cube, model, threshold=0.5, device=None)
            if label==1 :
                faux_positif.append(cube)
                probas_faux_positif.append(prob)

        for cube in cubes["random"]:
            prob,label=predict_cube(cube, model, threshold=0.5, device=None)
            if label==1 :
                faux_positif.append(cube)
                probas_faux_positif.append(prob)
    print(f'nombre de faux_negatifs : {len(faux_negatif)}')
    print(f'nombre de faux_positifs : {len(faux_positif)}')
    return faux_negatif,probas_faux_negatif,faux_positif,probas_faux_positif

## **Sur un df négatif**

In [ ]:
def hard_negatives(df):
    hard_negatives=[]
    for index,row in tqdm(df.iterrows(),total=len(df)):
        patient_path=os.path.join(series_path,
                                      f"{df['SeriesInstanceUID'][index]}")
        volume=preprocessing(patient_path)
        liste_cubes=get_non_overlapping_cubes(volume, cube_size=48)

        for cube in liste_cubes:
            result=predict_cube(cube, model, threshold=0.5, device=device)
            if result[1]==1 :
                hard_negatives.append(cube)

    return hard_negatives

# **9. Entrainement d'un modèle**

In [ ]:
# === boucle d'entraînement ===
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, device="cuda"):


    model.to(device)
    best_val_acc =0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}")

        if val_loader is not None:
            val_loss, val_acc, val_positions_loss = evaluate_model(model, val_loader, criterion, device)
            print(f"         Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Positions BCE: {val_positions_loss:.4f}")

             # ----------- Sauvegarde si meilleure val_acc -----------
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), save_path)
                print(f"         -> Nouveau meilleur modèle sauvegardé ! (Val Acc: {best_val_acc:.4f})")

    return model

# **10. Evaluation d'un modèle**

In [ ]:
import torch.nn.functional as F

def combined_loss(pred, target, alpha=0.1):
    pos_pred = torch.sigmoid(pred[:, :13])   # si sortie logit
    pos_target = target[:, :13]
    label_pred = pred[:, 13:]
    label_target = target[:, 13:]

    loss_pos = F.binary_cross_entropy(pos_pred, pos_target)
    loss_label = F.binary_cross_entropy_with_logits(label_pred, label_target)

    return alpha * loss_pos + loss_label

In [ ]:
# === fonction d'évaluation (validation/test) ===
import torch
import torch.nn.functional as F

def evaluate_model(model, loader, criterion, device="cuda"):
    """
    Évalue le modèle sur un loader donné.
    Loss combinée = BCE(label) + alpha * BCE(positions)
    Retourne :
      - avg_loss : loss totale combinée
      - accuracy_label : précision sur le label binaire
      - avg_positions_loss : BCE moyenne sur les positions (0 à 12)
    """
    model.eval()
    running_loss = 0.0
    correct_label = 0
    positions_loss_total = 0.0  # pour BCE positions

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)  # (B,14)

            # Calcul de la loss combinée via la fonction criterion
            loss = criterion(outputs, labels,alpha=0.1)
            running_loss += loss.item() * inputs.size(0)

            # Accuracy binaire pour le label (colonne 13)
            probs_label = torch.sigmoid(outputs[:, -1])
            best_thresh = 0.65
            preds_label = (probs_label > best_thresh).float()
            labels_binary = labels[:, -1]
            correct_label += (preds_label == labels_binary).sum().item()

            # BCE positions (colonnes 0 à 12)
            pos_pred = torch.sigmoid(outputs[:, :13])
            pos_target = labels[:, :13]
            bce_positions = F.binary_cross_entropy(pos_pred, pos_target, reduction='sum')
            positions_loss_total += bce_positions.item()

    n_samples = len(loader.dataset)
    avg_loss = running_loss / n_samples
    accuracy_label = correct_label / n_samples
    avg_positions_loss = positions_loss_total / n_samples

    return avg_loss, accuracy_label, avg_positions_loss

# **11. Trouver les meilleurs paramètres**

In [ ]:
def find_best_threshold(model, loader, device="cuda", thresholds=np.arange(0.1, 0.91, 0.05)):
    model.eval()
    all_probs = []
    all_labels = []

    # Récupérer toutes les probabilités et labels
    with torch.no_grad():
        for inputs, labels in loader:

            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)
            probs = torch.sigmoid(outputs[:, -1])  # dernière colonne = label binaire
            all_probs.append(probs.cpu())
            all_labels.append(labels[:, -1].cpu())

    all_probs = torch.cat(all_probs)
    all_labels = torch.cat(all_labels)

    best_acc = 0
    best_thresh = 0
    acc_list = []

    # Tester tous les seuils

    for t in tqdm(thresholds, desc="Testing thresholds"):
        preds = (all_probs > t).float()
        acc = (preds == all_labels).float().mean().item()
        acc_list.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_thresh = t

    # Afficher la courbe Accuracy vs Threshold
    plt.figure(figsize=(8,5))
    plt.plot(thresholds, acc_list, marker='o')
    plt.xlabel("Seuil")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Threshold pour le label anévrisme")
    plt.grid(True)
    plt.show()

    print(f"Meilleur seuil : {best_thresh:.2f}, Accuracy : {best_acc:.4f}")
    return best_thresh, best_acc

In [ ]:
%%writefile utils.py

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from scipy.ndimage import rotate, shift,zoom , map_coordinates, gaussian_filter
import random
import os
import glob
import ast

import matplotlib.pyplot as plt
import pydicom

from tqdm import tqdm

def get_instance_number(series_path,patient_path,df_loc):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]
    exemple_path = os.path.join(series_path,
                                  f"{numero_patient}/{numero_coupe}.dcm")
    #print('Récupération des coordonnées du voxel...')
    ds = pydicom.dcmread(exemple_path)

    InstanceNumber=ds.InstanceNumber

    return int(InstanceNumber)

def coordonnee_z(patient_path,InstanceNumber=163):

    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]
    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))

    # Trouver l’indice z dans le volume
    z_index = [i for i, s in enumerate(slices) if int(s.InstanceNumber) == InstanceNumber][0]

    return z_index


def get_center(series_path,patient_path,df_loc):

    numero_patient=os.path.basename(patient_path)
    numero_coupe=df_loc[df_loc['SeriesInstanceUID']==numero_patient]['SOPInstanceUID'].iloc[0]

    InstanceNumber=get_instance_number(series_path,patient_path,df_loc)
    z=coordonnee_z(patient_path,InstanceNumber)

    coord_str = df_loc[df_loc['SOPInstanceUID'] == numero_coupe]['coordinates'].iloc[0]
    coord_dict = ast.literal_eval(coord_str)
    x = coord_dict['y']
    y = coord_dict['x'] #fait exprès

    center=np.array([x,y,z])
    return center


def get_pixelspacing(path):
    """
    Récupère l'espacement voxel (mm) depuis un fichier DICOM.
    Retourne (row_spacing, col_spacing, slice_thickness)
    """
    dcm = pydicom.dcmread(path)

    # Espacement dans le plan (mm/pixel)
    if "PixelSpacing" in dcm:
        row_spacing, col_spacing = [float(x) for x in dcm.PixelSpacing]
    else:
        row_spacing, col_spacing = None, None

    # Épaisseur de coupe (mm)
    slice_thickness = float(getattr(dcm, "SliceThickness", 1.0))

    return row_spacing, col_spacing, slice_thickness

def get_patient_ID(patient_path):
    return str(os.path.basename(patient_path))


def get_position(df_train,patient_path):
    positions=['Left Infraclinoid Internal Carotid Artery',
       'Right Infraclinoid Internal Carotid Artery',
       'Left Supraclinoid Internal Carotid Artery',
       'Right Supraclinoid Internal Carotid Artery',
       'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery',
       'Anterior Communicating Artery', 'Left Anterior Cerebral Artery',
       'Right Anterior Cerebral Artery', 'Left Posterior Communicating Artery',
       'Right Posterior Communicating Artery', 'Basilar Tip',
       'Other Posterior Circulation']
    row=df_train[df_train["SeriesInstanceUID"] == get_patient_ID(patient_path)][positions].values.flatten()
    return row


def show_middle_slices(volume):

    # volume shape : (X, Y, Z)

    mid_x = volume.shape[0] // 2
    mid_y = volume.shape[1] // 2

    mid_z = volume.shape[2] // 2


    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    # Coupe axiale (XY plane à profondeur z)
    axes[0].imshow(volume[:, :, mid_z].T, cmap='gray')  # transpose pour que X horizontal, Y vertical
    axes[0].set_title(f'Axiale (z={mid_z})')
    axes[0].axis('on')

    # Coupe coronale (XZ plane à coordonnée y)
    axes[1].imshow(volume[:, mid_y, :].T, cmap='gray')  # transpose pour X horizontal, Z vertical
    axes[1].set_title(f'Coronale (y={mid_y})')
    axes[1].axis('on')

    # Coupe sagittale (YZ plane à coordonnée x)
    axes[2].imshow(volume[mid_x, :, :].T, cmap='gray')  # transpose pour Y horizontal, Z vertical
    axes[2].set_title(f'Sagittale (x={mid_x})')
    axes[2].axis('on')

    plt.tight_layout()
    plt.show()


def show_slice_with_point(volume, coord, plane="axial"):
    x, y, z = coord.astype(int)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    if plane == "axial":       # plan XY à profondeur z
        img = volume[:, :, z].T
        ax[0].imshow(img, cmap="gray")

        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(x, y, c="r", s=40, marker="x")
        title = f"Axial z={z}"

    elif plane == "sagittal":  # plan YZ à abscisse x
        img = volume[x, :, :].T
        ax[0].imshow(img, cmap="gray")
        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(y, z, c="r", s=40, marker="x")

        title = f"Sagittal x={x}"

    elif plane == "coronal":   # plan XZ à ordonnée y
        img = volume[:, y, :].T
        ax[0].imshow(img, cmap="gray")
        ax[1].imshow(img, cmap="gray")
        ax[1].scatter(x, z, c="r", s=40, marker="x")
        title = f"Coronal y={y}"

    else:
        raise ValueError("plane doit être 'axial', 'sagittal' ou 'coronal'")

    for a in ax:
        a.set_title(title)
        a.axis("on")
    plt.tight_layout()
    plt.show()


def ajouter_Modality(df_main,df_info):
    df_merged = df_main.merge(
        df_info[['SeriesInstanceUID', 'Modality']],
        on='SeriesInstanceUID',
        how='left'  # conserve toutes les lignes de df_main
    )
    return df_merged


def dicom_to_numpy(patient_path):

    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]

    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))



    # On empile les pixel_array en un volume 3D NumPy (X,Y,Z)
    target_shape = slices[0].pixel_array.shape

    slices = [s for s in slices if s.pixel_array.shape == target_shape]
    volume = np.stack([s.pixel_array for s in slices], axis=-1)

    # Récupération du spacing réel
    pixel_spacing = slices[0].PixelSpacing
    dx, dy = pixel_spacing
    dz = getattr(slices[0], 'SliceThickness', 1.0)  # fallback si manquant

    return volume, (dx,dy,dz)

def resample(volume, spacing, target_spacing=(0.4, 0.4, 0.4)):
    zoom_factors = [s / t for s, t in zip(spacing, target_spacing)]
    new_volume = zoom(volume, zoom_factors, order=1)
    return new_volume


def crop(volume):

    # On crée un masque des voxels non nuls
    mask = volume > (volume.max() * 0.1)
    if not mask.any():
        return volume  # rien à couper

    # On récupère les indices min/max pour chaque dimension
    x_min, x_max = mask.any(axis=(1,2)).nonzero()[0][[0, -1]] #axe_x
    y_min, y_max = mask.any(axis=(0,2)).nonzero()[0][[0, -1]] #axe_y
    z_min, z_max = mask.any(axis=(0,1)).nonzero()[0][[0, -1]] #axe_z

    # Crop
    cropped = volume[x_min:x_max+1, y_min:y_max+1, z_min:z_max+1]
    return cropped, (x_min,y_min,z_min)

def normalization(volume):

    v_min, v_max = volume.min(), volume.max()
    if v_max > v_min:  # éviter la division par zéro
        volume = (volume - v_min) / (v_max - v_min)
    else:
        volume = np.zeros_like(volume)
    return volume


def resample_coordonnees(spacing, coords, target_spacing=0.4):
    x,y,z=coords
    if isinstance(target_spacing, (int, float)):
        target_spacing = (target_spacing, target_spacing, target_spacing)

    # coord physiques
    coords_mm = np.array([x*spacing[0], y*spacing[1], z*spacing[2]])
    # nouveaux indices
    new_voxel = coords_mm / np.array(target_spacing)
    return new_voxel


def preprocessing_volume_and_coords(series_path,patient_path,df_loc):

    coords=get_center(series_path,patient_path,df_loc)
    volume,spacing = dicom_to_numpy(patient_path)

    resample_volume = resample(volume, spacing,target_spacing=(0.4, 0.4, 0.4))
    resample_coords=resample_coordonnees(spacing, coords,target_spacing=0.4)

    crop_volume, crop_indices = crop(resample_volume)
    crop_coords=resample_coords - crop_indices

    norm_volume = normalization(crop_volume)

    return norm_volume,crop_coords


## Pour la fonction d'inférence
def preprocessing_volume(patient_path):

    volume,spacing = dicom_to_numpy(patient_path)
    resample_volume = resample(volume, spacing,target_spacing=(0.4, 0.4, 0.4))
    crop_volume, crop_indices = crop(resample_volume)
    new_volume = normalization(crop_volume)

    return new_volume


def random_deformation(volume, grid_size=3, max_displacement=3):


    shape = volume.shape
    assert len(shape) == 3, "Le volume doit être 3D"


    # Coordonnées originales du volume
    dz, dy, dx = shape
    z, y, x = np.meshgrid(
        np.linspace(0, dz-1, shape[0]),
        np.linspace(0, dy-1, shape[1]),
        np.linspace(0, dx-1, shape[2]),
        indexing="ij"
    )

    # Génère une grille de points de contrôle (par ex. 3x3x3)
    grid_z = np.linspace(0, dz-1, grid_size)
    grid_y = np.linspace(0, dy-1, grid_size)
    grid_x = np.linspace(0, dx-1, grid_size)
    control_points = np.meshgrid(grid_z, grid_y, grid_x, indexing="ij")

    # Crée un champ de déplacements aléatoires
    # (même taille que la grille de contrôle)
    displacement = [

        np.random.uniform(-max_displacement, max_displacement, size=(grid_size,grid_size,grid_size))
        for _ in range(3)
    ]

    # Interpolation spline cubique du champ de déplacement

    from scipy.interpolate import RegularGridInterpolator
    disp_interp = [
        RegularGridInterpolator((grid_z, grid_y, grid_x), d, bounds_error=False, fill_value=0)
        for d in displacement
    ]

    # Calcule le champ de coordonnées déformées
    coords = np.array([z, y, x])
    dz_new = disp_interp[0](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)
    dy_new = disp_interp[1](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)
    dx_new = disp_interp[2](np.array([z.flatten(), y.flatten(), x.flatten()]).T).reshape(shape)

    z_new = z + dz_new
    y_new = y + dy_new
    x_new = x + dx_new

    # Applique la déformation au volume
    deformed = map_coordinates(volume, [z_new, y_new, x_new], order=3, mode='reflect')

    return deformed


def data_augmentation(volume,n_aug):

    augmented_volumes = []
    for i in range(n_aug):
        vol_def = random_deformation(volume)
        augmented_volumes.append(vol_def)

    augmented_volumes = np.stack(augmented_volumes)  # shape = (12, 48, 48, 48)
    return augmented_volumes


def dataset_augmented(liste_volumes,n_aneurysm):
    liste_cubes = []

    for i in tqdm(range(len(liste_volumes))):
        volume = liste_volumes[i]  # shape (48,48,48)

        # data_augmentation doit renvoyer (n, 48, 48, 48)
        aug = data_augmentation(volume,n_aneurysm)

        liste_cubes.append(aug)

    # Concaténer tous les résultats en un seul tableau
    liste_cubes = np.concatenate(liste_cubes, axis=0)  # shape (N*n, 48, 48, 48)
    return liste_cubes


def extract_positives_cubes(volume, aneurysm_coords, N=10, cube_size=48, max_shift=7):

    cubes = []
    x_c, y_c, z_c = aneurysm_coords
    x_c, y_c, z_c = int(x_c), int(y_c), int(z_c)

    # Dimensions du volume
    D, H, W = volume.shape

    for _ in range(N):
        # Déplacement aléatoire autour de l'anévrisme
        dx = np.random.randint(-max_shift, max_shift+1)

        dy = np.random.randint(-max_shift, max_shift+1)
        dz = np.random.randint(-max_shift, max_shift+1)

        # Centre du cube
        cx, cy, cz = x_c + dx, y_c + dy, z_c + dz


        # Limiter les bornes pour rester dans le volume
        x_start = max(0, cx - cube_size//2)
        y_start = max(0, cy - cube_size//2)
        z_start = max(0, cz - cube_size//2)

        x_end = min(D, x_start + cube_size)
        y_end = min(H, y_start + cube_size)
        z_end = min(W, z_start + cube_size)

        # Ajuster le start si on est proche de la limite
        x_start = x_end - cube_size
        y_start = y_end - cube_size
        z_start = z_end - cube_size

        # Extraire le cube
        cube = volume[x_start:x_end, y_start:y_end, z_start:z_end]
        cubes.append(cube)

    return np.array(cubes)  # shape : (N, cube_size, cube_size, cube_size)

import numpy as np
def extract_negative_cubes(
    volume,
    aneurysm_coords,
    N: int = 20,
    cube_size: int = 48,
    min_distance: int = 75,
    exp_scale: float = 5.0,
    max_trials: int = 10_000):


    D, H, W = volume.shape
    half = cube_size // 2
    x_c, y_c, z_c = map(float, aneurysm_coords)

    cubes = []
    centers = []
    trials = 0

    while len(cubes) < N and trials < max_trials:
        trials += 1


        # Tirer une distance radiale : min_distance + Exp(scale)
        r = min_distance + np.random.exponential(exp_scale)

        # Tirer direction aléatoire sur la sphère
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        dx = r * np.sin(theta) * np.cos(phi)
        dy = r * np.sin(theta) * np.sin(phi)
        dz = r * np.cos(theta)

        cx = int(round(x_c + dx))
        cy = int(round(y_c + dy))
        cz = int(round(z_c + dz))


        # Vérifier que le cube tient dans le volume
        if (half <= cx < D - half and
            half <= cy < H - half and
            half <= cz < W - half):


            cube = volume[
                cx - half : cx + half,
                cy - half : cy + half,
                cz - half : cz + half
            ]

            cubes.append(cube)
            centers.append((cx, cy, cz))

    if len(cubes) < N:
        raise RuntimeError(
            f"Seulement {len(cubes)} cubes trouvés après {trials} essais."
        )

    return np.stack(cubes), centers


def dictionnaire_patient(series_path,patient_path,df,df_train,N_pos,N_neg):
    data={}

    volume,aneurysm_coords=preprocessing_volume_and_coords(series_path,patient_path,df)

    positive_cubes=extract_positives_cubes(volume, aneurysm_coords, N=N_pos, cube_size=48, max_shift=20)
    negative_cubes= extract_negative_cubes(volume,aneurysm_coords,N=N_neg,cube_size=48,min_distance=75,exp_scale=5.0)[0]


    position_single = get_position(df_train,patient_path).reshape(1,13)

    cubes = np.concatenate([positive_cubes,negative_cubes],axis=0)
    patient_ID = np.array([get_patient_ID(patient_path)] * (positive_cubes.shape[0] + negative_cubes.shape[0])).reshape(-1, 1)
    labels = np.concatenate([np.ones((positive_cubes.shape[0], 1), dtype=int), np.zeros((negative_cubes.shape[0], 1), dtype=int)])
    positions = np.concatenate([
        np.repeat(position_single,positive_cubes.shape[0] , axis=0),
        np.zeros((negative_cubes.shape[0], 13))
    ],axis=0)

    # Créer le dictionnaire par patient
    data_patient = {
        "cubes": cubes,
        "patient_ID": patient_ID,
        "labels": labels,
        "positions": positions
    }
    return data_patient


def build_dataset_dict(series_path, df, df_train,N_pos,N_neg):
    from tqdm import tqdm

    data = {}

    for i in tqdm(range(len(df))):
        series_uid = df.iloc[i]['SeriesInstanceUID']
        patient_path = os.path.join(series_path, series_uid)

        try:
            data[f'patient_{i}'] = dictionnaire_patient(series_path, patient_path, df, df_train,N_pos,N_neg)
        except Exception as e:
            print(f"Erreur pour {series_uid}: {e}")
            continue  # passer au patient suivant

    return data

def get_non_overlapping_cubes(volume, cube_size=48):

    z_dim, y_dim, x_dim = volume.shape
    cubes = []

    # parcours par pas de cube_size
    for zi in range(0, z_dim - cube_size + 1, cube_size):
        for yi in range(0, y_dim - cube_size + 1, cube_size):
            for xi in range(0, x_dim - cube_size + 1, cube_size):
                cube = volume[zi:zi+cube_size, yi:yi+cube_size, xi:xi+cube_size]
                cubes.append(cube)

    return cubes


# === boucle d'entraînement ===
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, device="cuda"):


    model.to(device)
    best_val_acc =0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            inputs = inputs.to(device)

            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}")

        if val_loader is not None:
            val_loss, val_acc, val_positions_loss = evaluate_model(model, val_loader, criterion, device)
            print(f"         Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Positions BCE: {val_positions_loss:.4f}")

             # ----------- Sauvegarde si meilleure val_acc -----------
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), save_path)
                print(f"         -> Nouveau meilleur modèle sauvegardé ! (Val Acc: {best_val_acc:.4f})")

    return model


import torch.nn.functional as F

def combined_loss(pred, target, alpha=0.1):
    pos_pred = torch.sigmoid(pred[:, :13])   # si sortie logit
    pos_target = target[:, :13]
    label_pred = pred[:, 13:]
    label_target = target[:, 13:]

    loss_pos = F.binary_cross_entropy(pos_pred, pos_target)
    loss_label = F.binary_cross_entropy_with_logits(label_pred, label_target)

    return alpha * loss_pos + loss_label


def evaluate_model(model, loader, criterion, device="cuda"):
    """
    Évalue le modèle sur un loader donné.
    Loss combinée = BCE(label) + alpha * BCE(positions)
    Retourne :
      - avg_loss : loss totale combinée
      - accuracy_label : précision sur le label binaire
      - avg_positions_loss : BCE moyenne sur les positions (0 à 12)
    """
    model.eval()
    running_loss = 0.0
    correct_label = 0
    positions_loss_total = 0.0  # pour BCE positions


    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)  # (B,14)

            # Calcul de la loss combinée via la fonction criterion
            loss = criterion(outputs, labels,alpha=0.1)
            running_loss += loss.item() * inputs.size(0)

            # Accuracy binaire pour le label (colonne 13)
            probs_label = torch.sigmoid(outputs[:, -1])
            best_thresh = 0.65
            preds_label = (probs_label > best_thresh).float()
            labels_binary = labels[:, -1]
            correct_label += (preds_label == labels_binary).sum().item()

            # BCE positions (colonnes 0 à 12)
            pos_pred = torch.sigmoid(outputs[:, :13])
            pos_target = labels[:, :13]
            bce_positions = F.binary_cross_entropy(pos_pred, pos_target, reduction='sum')
            positions_loss_total += bce_positions.item()

    n_samples = len(loader.dataset)
    avg_loss = running_loss / n_samples
    accuracy_label = correct_label / n_samples
    avg_positions_loss = positions_loss_total / n_samples

    return avg_loss, accuracy_label, avg_positions_loss


def find_best_threshold(model, loader, device="cuda", thresholds=np.arange(0.1, 0.91, 0.05)):
    from tqdm import tqdm

    model.eval()

    all_probs = []
    all_labels = []

    # Récupérer toutes les probabilités et labels
    with torch.no_grad():
        for inputs, labels in loader:

            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)
            probs = torch.sigmoid(outputs[:, -1])  # dernière colonne = label binaire
            all_probs.append(probs.cpu())
            all_labels.append(labels[:, -1].cpu())

    all_probs = torch.cat(all_probs)
    all_labels = torch.cat(all_labels)

    best_acc = 0
    best_thresh = 0
    acc_list = []

    # Tester tous les seuils


    for t in tqdm(thresholds, desc="Testing thresholds"):
        preds = (all_probs > t).float()
        acc = (preds == all_labels).float().mean().item()
        acc_list.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_thresh = t

    # Afficher la courbe Accuracy vs Threshold
    plt.figure(figsize=(8,5))
    plt.plot(thresholds, acc_list, marker='o')
    plt.xlabel("Seuil")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Threshold pour le label anévrisme")
    plt.grid(True)
    plt.show()

    print(f"Meilleur seuil : {best_thresh:.2f}, Accuracy : {best_acc:.4f}")
    return best_thresh, best_acc

Writing utils.py
